In [1]:
import torch
from datasets import load_dataset
import random


#Get the data from hugginface
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)

        def f(data):
            # only keep the senten length > 40
            return len(data['text']) > 40

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        # set the 1st sentence 0:20, 2nd sentence 20:40
        sentence1 = text[:20]
        sentence2 = text[20:40]
        label = 0

        # 50% chance get the no relationship data
        if random.randint(0, 1) == 0:
            # j => random index of the sentence
            j = random.randint(0, len(self.dataset) - 1)
            # get a pair of no relationship sentence, set label to 1
            sentence2 = self.dataset[j]['text'][20:40]
            label = 1

        return sentence1, sentence2, label


dataset = Dataset('train')

sentence1, sentence2, label = dataset[0]

len(dataset), sentence1, sentence2, label

Found cached dataset parquet (C:/Users/35391/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Filter:   0%|          | 0/9600 [00:00<?, ? examples/s]

(8001, '选择珠江花园的原因就是方便，有电动扶梯直', '接到达海边，周围餐馆、食廊、商场、超市、', 0)

In [6]:
sentence1, sentence2, label = dataset[4]

len(dataset), sentence1, sentence2, label

(8001, '机器背面似乎被撕了张什么标签，残胶还在。', '一读难忘，欲罢不能。现代的都市生活，每天', 1)

In [25]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# use to(device) set the data and model to use cuda
device

device(type='cuda', index=0)

In [8]:
from transformers import BertTokenizer

# get the chinese version tokenizer
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [17]:
def collate_fn(data):
    # sents = [(sent1,sent2).....]
    sents = [i[:2] for i in data]
    labels = [i[2] for i in data]

    # encode the data
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=45,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    #input_ids:map the id to different words
    #attention_mask: words == 1, padding  == 0 [1,1,1,1,1,0,0]
    #token_type_ids: use for 1st sentence and 2nd sentence [0,0,0,0,0,1,1,1,1,1,1,1]
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


# put the data into torch dataloader
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    print(len(loader))
    for idx in range(len(input_ids)):
        print(token.decode(input_ids[idx]))
    print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels)
    brea

1000
[CLS] 订 房 时 号 称 是 海 景 房, 结 果 入 住 是 3 楼, 别 说 [SEP] 海 景, 就 是 旁 边 的 绿 博 院 也 看 不 到. 好 在 白 天 [SEP] [PAD] [PAD]
[CLS] 同 样 价 格 的 行 政 楼 层 房 间 有 大 有 小 ， 需 要 注 意 [SEP] 。 洗 手 间 装 修 布 置 的 很 舒 适 漂 亮 ， 有 淋 浴 和 浴 [SEP] [PAD] [PAD]
[CLS] 生 下 孩 子 后 就 买 了 这 本 书 ， 觉 得 实 在 是 相 见 恨 [SEP] 晚 啊! 我 的 身 体 在 怀 孕 前 就 比 较 差 ， 也 是 辗 转 [SEP] [PAD] [PAD]
[CLS] 这 是 我 第 一 次 通 过 当 当 网 买 书 ， 朋 友 说 这 里 买 [SEP] 我 非 常 不 能 理 解 的 是 ， 300 多 一 晚 的 酒 店 居 [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] 我 也 是 看 了 评 论 才 买 的 ， 一 买 回 来 ， 儿 子 就 拿 [SEP] 起 荒 岛 历 险 来 看 ， 非 常 感 兴 趣 ， 问 他 为 什 么 好 [SEP] [PAD] [PAD]
[CLS] 也 许 期 望 过 高 ， 拿 到 此 说 后 ， 读 了 一 部 分 觉 得 [SEP] ！ 做 工 十 分 精 致 ， 很 小 很 轻 便 ， 携 带 方 便 ！ 风 [SEP] [PAD] [PAD]
[CLS] 在 同 档 次 的 笔 记 本 电 脑 中 性 价 比 高 ， 配 置 也 够 [SEP] 用 ， 品 牌 效 应 好 ， 外 观 不 错 ， 已 在 使 用 中 待 发 [SEP] [PAD] [PAD]
[CLS] 开 机 20 分 钟 触 摸 板 能 烫 伤 手 指 ， 仔 细 检 查 风 [SEP] 看 完 。 当 时 我 的 奶 不 是 很 多 ， 宝 宝 总 要 吃 ， 家 [SEP] [PAD] [PAD] [PAD]
torch.Size([8, 45]) torch.Size([8, 45]) torch.Size([8, 45]) tensor([0, 0, 0, 1, 0, 1, 0, 1])


In [27]:
from transformers import BertModel

# get the pretrained model of bert, need the same version of the tokenizer
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

# don't need train the model and update the params
for param in pretrained.parameters():
    param.requires_grad_(False)

# try the first batch, last_hidden_state will give the model's output
out = pretrained(input_ids=input_ids.to(device),
           attention_mask=attention_mask.to(device),
           token_type_ids=token_type_ids.to(device))

out.last_hidden_state.shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([8, 45, 768])

In [29]:
# define forward model
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # set the Fully Connected Layer(fc layer)
        # output willbe the softmax(token.vocab_size) which size should be 2(0/1)
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            # do not need update the weight
            out = pretrained(input_ids=input_ids.to(device),
                             attention_mask=attention_mask.to(device),
                             token_type_ids=token_type_ids.to(device))
        # get the softmax value of the output
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)

        return out


model = Model().to(device)

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([8, 2])

In [32]:
from transformers import AdamW

# train optimizer = AdamW
optimizer = AdamW(model.parameters(), lr=5e-4)
# define the loss
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels.to(device))
    # backforward
    loss.backward()
    # use the optimizer
    optimizer.step()
    # reset the optimizer step to zero, can be put in front of backward
    optimizer.zero_grad()

    if i % 100 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels.to(device)).sum().item() / len(labels.to(device))
        print(i, loss.item(), accuracy)


0 0.5043620467185974 0.875
100 0.62580806016922 0.625
200 0.5098405480384827 0.75
300 0.36935633420944214 1.0
400 0.33695387840270996 1.0
500 0.4939813017845154 0.875
600 0.5929908752441406 0.75
700 0.4678238332271576 0.875
800 0.4044565260410309 1.0
900 0.3614732623100281 1.0


In [38]:
# test the model
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

#         if i == 5:
#             break


        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        pred = out.argmax(dim=1)
        
        if i<5:
            print(i)
            print(pred)
            print(labels)
            print(pred == labels.to(device))
        else:
            if i % 10 == 0:
                print(i)

        correct += (pred == labels.to(device)).sum().item()
        total += len(labels)

    print(correct / total)


test()

Found cached dataset parquet (C:/Users/35391/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached processed dataset at C:\Users\35391\.cache\huggingface\datasets\lansinuote___parquet\lansinuote--ChnSentiCorp-4d058ef86e3db8d5\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-680cb52f3e4569f7.arrow


0
tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 1, 0], device='cuda:0')
tensor([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0])
tensor([ True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False, False,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True], device='cuda:0')
1
tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 0])
tensor([ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False,  True,  True,  True,
        False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
  